In [ ]:
# WHAT: Load the union master; verify schema and coverage
from google.colab import drive
drive.mount('/content/drive')

import os, pandas as pd, numpy as np
BASE = "/content/drive/MyDrive/MLBA_Project"
RAW  = f"{BASE}/data/raw"
PRO  = f"{BASE}/data/processed"
os.makedirs(PRO, exist_ok=True)

df = pd.read_csv(f"{PRO}/clean_master_union.csv", parse_dates=["date"])
df.columns = df.columns.str.lower()
df = df.sort_values(["fund_id","date"]).reset_index(drop=True)

req = {"date","fund_id","nav","tri","india_vix","usd_inr","gsec_10y_yield","gold_inr","brent_usd"}
missing = [c for c in req if c not in df.columns]
print("Rows:", len(df), "| funds:", df["fund_id"].nunique(),
      "| range:", df["date"].min().date(), "→", df["date"].max().date())
print("Missing (ok if some macros were not provided):", missing)

In [ ]:
# WHAT: Daily % returns and multi-horizon returns/excess
df["ret_1d"] = df.groupby("fund_id")["nav"].pct_change()
df["bmk_1d"] = df["tri"].pct_change()

horizons = [7,14,21,63]
for h in horizons:
    df[f"ret_{h}d"] = df.groupby("fund_id")["nav"].pct_change(h)
    df[f"bmk_{h}d"] = df["tri"].pct_change(h)
    df[f"excess_{h}d"] = df[f"ret_{h}d"] - df[f"bmk_{h}d"]

print("Added daily & multi-horizon momentum/excess.")

In [ ]:
# WHAT: Rolling volatility & Sharpe (126d)
import numpy as np
rf_annual = 0.06
rf_daily  = rf_annual / 252.0
win = 126

roll_std  = df.groupby("fund_id")["ret_1d"].rolling(win).std().reset_index(level=0, drop=True)
roll_mean = df.groupby("fund_id")["ret_1d"].rolling(win).mean().reset_index(level=0, drop=True)

df["vol_126d"]    = roll_std * np.sqrt(252)
den = roll_std.replace(0, np.nan)
df["sharpe_126d"] = (roll_mean - rf_daily) / den
print("Added vol_126d, sharpe_126d.")

In [ ]:
# WHAT: Rolling beta vs TRI and annualized alpha (126d)
win_beta = 126
def beta_alpha_block(g):
    r = g["ret_1d"]; m = g["bmk_1d"]
    cov = r.rolling(win_beta).cov(m)
    var = m.rolling(win_beta).var()
    beta = cov / var.replace(0, np.nan)
    mean_r = r.rolling(win_beta).mean()
    mean_m = m.rolling(win_beta).mean()
    alpha_daily = (mean_r - rf_daily) - beta * (mean_m - rf_daily)
    alpha_ann = alpha_daily * 252
    return pd.DataFrame({"beta_126d": beta, "alpha_ann_126d": alpha_ann})

ba = df.groupby("fund_id", group_keys=False).apply(beta_alpha_block)
df[["beta_126d","alpha_ann_126d"]] = ba.values
print("Added beta_126d, alpha_ann_126d.")

In [ ]:
# WHAT: Rolling 1y max drawdown and % positive days over 63d
win_dd, win_cons = 252, 63

roll_peak = (df.groupby("fund_id")["nav"]
               .rolling(win_dd, min_periods=1).max()
               .reset_index(level=0, drop=True))
dd = df["nav"] / roll_peak - 1.0

mdd = (dd.groupby(df["fund_id"])
         .rolling(win_dd, min_periods=1).min()
         .reset_index(level=0, drop=True))
df["mdd_252d"] = mdd.values

is_pos = (df["ret_1d"] > 0).astype(float)
cons = (is_pos.groupby(df["fund_id"])
              .rolling(win_cons, min_periods=1).mean()
              .reset_index(level=0, drop=True))
df["consistency_pos_63d"] = cons.values

print("Added mdd_252d, consistency_pos_63d.")

In [ ]:
# WHAT: Rolling correlation with benchmark and simple regime flags (63d)
win_reg = 63
def rolling_corr(g):
    return g["ret_1d"].rolling(win_reg).corr(g["bmk_1d"])

df["corr_bmk_63d"] = df.groupby("fund_id", group_keys=False).apply(rolling_corr).values
df["regime_bull_63d"] = (df["bmk_63d"] > 0).astype(int)
df["regime_bear_63d"] = (df["bmk_63d"] < 0).astype(int)

print("Added corr_bmk_63d + regime flags.")

In [ ]:
# WHAT: Lag macro factors by 5/10 business days
macro_cols = [c for c in ["india_vix","usd_inr","gsec_10y_yield","gold_inr","brent_usd"] if c in df.columns]
for c in macro_cols:
    df[f"{c}_lag5"]  = df[c].shift(5)
    df[f"{c}_lag10"] = df[c].shift(10)
print("Added macro lags for:", macro_cols)

In [ ]:
# WHAT: Cross-sectional ranks per date
def xs_rank(col, asc=False):
    return df.groupby("date")[col].rank(pct=True, ascending=asc)

if "excess_21d" in df.columns:
    df["xs_rank_excess_21d"]  = xs_rank("excess_21d", asc=False)
if "sharpe_126d" in df.columns:
    df["xs_rank_sharpe_126d"] = xs_rank("sharpe_126d", asc=False)
if "mdd_252d" in df.columns:
    df["xs_rank_mdd_252d"]    = xs_rank("mdd_252d", asc=True)

print("Added cross-section ranks (if available).")

In [ ]:
# WHAT: Drop sparse columns (>=80% NaN) and sparse rows (<90% filled)
col_keep = [c for c in df.columns if c not in ["fund_id","date"]]
col_na = df[col_keep].isna().mean()
cols_ok = ["fund_id","date"] + [c for c in col_keep if col_na[c] < 0.80]

df_feat = df[cols_ok].copy()
row_nonmiss = 1.0 - df_feat.isna().mean(axis=1)
df_feat = df_feat[row_nonmiss >= 0.90].copy()
df_feat = df_feat.sort_values(["fund_id","date"]).reset_index(drop=True)

out_features = f"{PRO}/clean_with_features.csv"
df_feat.to_csv(out_features, index=False)
print("Saved features →", out_features, "| rows:", len(df_feat), "| cols:", len(df_feat.columns))

In [ ]:
as_of = df_feat["date"].max()
latest = df_feat[df_feat["date"] == as_of].copy()

def safe_rank(s, asc=False): 
    return s.rank(pct=True, ascending=asc)

parts = []
if "excess_63d" in latest.columns:   parts.append(0.6 * safe_rank(latest["excess_63d"], asc=False))
if "excess_21d" in latest.columns:   parts.append(0.2 * safe_rank(latest["excess_21d"], asc=False))
if "sharpe_126d" in latest.columns:  parts.append(0.2 * safe_rank(latest["sharpe_126d"], asc=False))

if parts:
    latest["score"] = sum(parts)
    cols = ["fund_id","date","score"] + [c for c in ["excess_63d","excess_21d","sharpe_126d","vol_126d","mdd_252d"] if c in latest.columns]
    leaderboard = latest[cols].sort_values("score", ascending=False).reset_index(drop=True)
else:
    leaderboard = latest[["fund_id","date"]].copy()
    leaderboard["score"] = np.nan

leader_path = f"{PRO}/leaderboard.csv"
leaderboard.to_csv(leader_path, index=False)
print("Saved leaderboard →", leader_path, "| rows:", len(leaderboard), "| as-of:", as_of.date())

In [ ]:
schema = [{"column": c, "dtype": str(df_feat[c].dtype), "missing_pct": float(df_feat[c].isna().mean())} 
          for c in df_feat.columns]
dc = pd.DataFrame(schema).sort_values("missing_pct", ascending=False)
dc.to_csv(f"{PRO}/data_card.csv", index=False)
dc.head(10)